# [Dacon LG SYSTEM]

*KU LeeDongGyu*

# Contents
---

1. Data Preprocessing
```
1) Data Import
2) Data Augmentation
```
2. Support Functions & Almost Original Xception
```
1) Support Functions
2) Almost Original Xception
3) Xception Evaluate
```



# Install Packages


# Module

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/Colab Notebooks/DACON/LG_SYSTEM

/content/drive/My Drive/Colab Notebooks/DACON/LG_SYSTEM


In [9]:
#from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import os
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

current_day='tmp' # 오늘날짜
os.makedirs(current_day, exist_ok=True) # 폴더 생성
PATH = './data/' # 코랩에서 작업하므로, 초기경로 보충설정

# 1. Data Preprocessing
---

# 1) 데이터 불러오기

In [10]:
train_err = pd.read_csv(PATH+'train_err_data.csv')
train_quality = pd.read_csv(PATH+'train_quality_data.csv')
train_problem = pd.read_csv(PATH+'train_problem_data.csv')

test_err = pd.read_csv(PATH+'test_err_data.csv')
test_quality = pd.read_csv(PATH+'test_quality_data.csv')

sample_submssion = pd.read_csv(PATH+'sample_submission.csv')

In [11]:
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000


test_user_id_max = 30000
test_user_id_min = 49998
test_user_number = 14999

# 2) Training 전처리

## 2-1) model(fwver)변수와 err_type 변수 가공

In [12]:
# STEP 1. model(fwver)과 err_type 변수 사용

################################################# 아래의 둘 중 하나의 케이스를 사용하여 작업하자. #################################################

In [13]:
#######################################################################################################################################################################################################
########################## case2. 각 유저에 대해 에러가 발생한 횟수를 단순히 합한 경우: y의 카운트가 문제가 되지는 않지만, 어느 모델에서 문제가 생겼는지 혼란스러울 수 있다. ##########################
#######################################################################################################################################################################################################

In [14]:
train_err['errtype']=train_err.errtype.astype('object') # 더미변수 생성하기 위해 카테고리화
dumi=pd.get_dummies(train_err[['user_id','model_nm','errtype']]) # model_nm과 errtype에 대해 더미변수 생성 (user_id 사용이뉴는 dumi변수에 그대로 가져가기 위함.)
tmp1=dumi.groupby(['user_id']).sum().astype(int).reset_index()

def convert_1(x): 
  # 숫자가 있으면 1, 없으면 0으로 바꾸는 함수.
  if x>0:
    return int(1)
  else:
    return int(0)

# model 변수에 대해 16,554,663개가 15000개로 압축되어 count되었기 때문에, 이를 1 또는 0으로 변환.
# tmp1에 아래의 데이터프레임을 저장함.
for i in range(9):
  tmp1['model_nm_model_'+str(i)]=tmp1['model_nm_model_'+str(i)].apply(convert_1)

display(tmp1)

,user_id,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
0,10000,0,0,0,1,0,0,0,0,0,0,0,8,104,0,1,1,0,0,7,15,16,1,10,59,61,0,0,0,0,0,0,0,0,0,32,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10001,0,0,1,0,0,0,0,0,0,0,0,0,0,53,1,1,0,0,0,10,11,1,15,151,128,0,4,1,3,1,756,751,5,1,22,0,0,0,250,0,10,18,0,1,1,0,0,113,56,1
2,10002,0,0,0,1,0,0,0,0,0,0,0,2,132,1,2,1,0,0,1,13,14,1,4,52,52,0,0,0,0,0,0,0,2,0,25,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10003,0,0,1,0,0,0,0,0,0,0,0,0,0,2,1,1,0,0,0,9,9,0,0,52,30,1,0,0,0,0,28,19,0,0,59,0,0,0,65,0,8,0,0,1,1,2,0,17,1,0
4,10004,1,0,0,0,0,0,0,0,0,0,0,0,1,0,3,4,0,0,0,16,19,3,5,143,91,0,0,0,0,0,140,119,0,0,33,0,0,0,176,0,16,0,0,1,1,0,0,4,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,24995,0,0,1,0,0,0,0,0,0,0,0,0,0,2,5,5,0,0,0,5,10,8,8,26,18,0,0,0,0,0,17,12,0,0,22,0,0,0,31,0,5,0,0,0,0,0,0,9,7,4
14996,24996,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
14997,24997,1,0,0,0,0,0,0,0,0,0,0,0,1,8,1,1,0,0,0,16,18,4,1,181,138,0,0,0,0,0,50,29,0,0,49,0,0,0,223,0,16,17,0,1,1,0,0,58,8,5
14998,24998,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,15,0,1,51,12,0,0,0,0,0,7,1,0,0,4,0,0,0,14,0,15,12,0,1,1,0,0,6,0,0


In [15]:
################################################# 케이스 선택 끝. 최종적으로 이후 사용할 변수는    ==> tmp1 <==    변수임. #################################################

## 2-2) 가장 많은 error가 발생한 시점과, 그때의 error 발생 건수 가공

In [16]:
# STEP 2. 가장 많은 error가 발생한 시점과, 그때의 error 발생 건수.

################################################# 아래의 둘 중 하나의 케이스를 사용하여 작업하자. (위의 STEP1과 이어서 사용 할것.) #################################################

In [17]:
#######################################################################################################################################################################################################
########################## case2. 각 유저에 대해 에러가 발생한 횟수를 단순히 합한 경우: y의 카운트가 문제가 되지는 않지만, 어느 모델에서 문제가 생겼는지 혼란스러울 수 있다. ##########################
#######################################################################################################################################################################################################

In [18]:
# 시간 변환함수. (출처: 데이콘 baseline)
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    #hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year,month, day)

In [19]:
train_err['month_day']=train_err['time'].apply(make_datetime) # 우리는 월,일만 살펴본다.
train_err['cnt']=1 # 카운트를 세기위한 보조변수

In [20]:
tmp2= train_err[['user_id','month_day','cnt']].groupby(['user_id','month_day']).count()

del train_err['month_day'], train_err['cnt'] # 만들었던거 삭제

# user_id별 에러가 최대로 발생한 날과, 그 에러의 수. (모델과 관련 없이 에러를 카운트 하는 case2와 관련이 깊다.)
tmp2= tmp2.sort_values(by='cnt', ascending=False).reset_index().groupby(['user_id']).head(1).sort_values('user_id').reset_index(drop=True)
display(tmp2)

,user_id,month_day,cnt
0,10000,2020-11-07,20
1,10001,2020-11-27,1452
2,10002,2020-11-07,17
3,10003,2020-11-11,24
4,10004,2020-11-20,102
...,...,...,...
14995,24995,2020-11-27,30
14996,24996,2020-11-30,4
14997,24997,2020-11-04,64
14998,24998,2020-11-10,37


In [21]:
###################################################################### 케이스 선택 끝. ######################################################################

In [22]:
#########################################################################################################################################
####################################### 위의 케이스 둘 중 하나를 선택했으면, 아래에서 추가가공을 진행 #######################################
#########################################################################################################################################

In [23]:
################### 참고 ###################  
# 시작하기 앞서 test_err 데이터에 대해 잠깐 살펴보면,
# train_err 데이터의 기간은 12월 2일까지 약 1600만개의 데이터가있는 반면, test_err 데이터는 약 12월 14일까지 1600만개의 데이터가 있다. (출발시점은 10/31로 유사)
# 그리고 12월 3일 부터 12월 14일까지 test_err 의 발생 건수는 1600만개중 15개에불과하다.
# 만약, quality까지 날짜를 동일하게 하려면, 11월 30일까지만 잡는것도 고려대상이다. (12월 이후로는 test_err데이터든, train_err데이터든 error 발생 개수가 상대적으로 매우 적다.)

# 중요!) 매우 다행인점은, 우리가 최대 빈도수를 기준으로 시간 거리를 잡았기 때문에, 이러한 이상치에 영향을 받지 않는다. (타당한 접근임을 반증)
# 이에 따라 우리는 최대 살펴볼 요일범위를 11월 30일 23시 59분 59초로 정한다.

# 아래는 이에 대한 근거.


train_err['cnt']=1
train_err['make_time']=train_err['time'].apply(make_datetime)
temp1=train_err[['user_id','make_time','cnt']].groupby(['user_id','make_time']).count().sort_values('make_time')

del train_err['cnt'], train_err['make_time']
print('Train 시간 하위 50개')
display(temp1.tail(50))

test_err['cnt']=1
test_err['make_time']=test_err['time'].apply(make_datetime)
temp2=test_err[['user_id','make_time','cnt']].groupby(['user_id','make_time']).count().sort_values('make_time')

del test_err['cnt'], test_err['make_time']
print('Test 시간 하위 50개')
display(temp2.tail(50))

################### 참고 끝. ###################  

Train 시간 하위 50개


,,cnt
user_id,make_time,
24999,2020-11-30,17
21611,2020-11-30,10
21608,2020-11-30,20
21609,2020-11-30,8
12118,2020-11-30,15
12120,2020-11-30,12
12121,2020-11-30,9
21613,2020-11-30,14
21610,2020-11-30,21


Test 시간 하위 50개


,,cnt
user_id,make_time,
39088,2020-11-30,35
35505,2020-11-30,14
42982,2020-11-30,12
31883,2020-11-30,43
35478,2020-11-30,31
39089,2020-11-30,43
41859,2020-11-30,39
39118,2020-11-30,28
35504,2020-11-30,30


In [24]:
################################################################# 이것 까지 해야 완료. #################################################################


max_time = 20201130235959 # 최대시간을 11월 30일 23시 59분 59초라 설정해보자. (우리는 빈도로 접근하기 때문에, 최대값으로 12월이 넘어가는 애러발생시점이 나오지 않는다.)
tmp2['due']=make_datetime(max_time)
tmp2['valid_time']=(tmp2['due']-tmp2['month_day']).dt.days # 시간변수 추가.

del tmp2['month_day'], tmp2['due']
display(tmp2) # cnt변수는 에러가 최대로 발생한 시점의 에러 개수 // valid_time 변수는 에러가 최대로 발생한 시점으로부터 문의를 보낼수 있는 유효시간

,user_id,cnt,valid_time
0,10000,20,23
1,10001,1452,3
2,10002,17,23
3,10003,24,19
4,10004,102,10
...,...,...,...
14995,24995,30,3
14996,24996,4,0
14997,24997,64,26
14998,24998,37,20


In [25]:
################################################# 여기까지 하고 나서 나온 최종적인 변수는    ==> tmp2 <==    를 사용함. #################################################

## 2-3) quality 변수에 대한 가공

In [26]:
# STEP 3. err_code 변수에 대한 전처리


######################################################################################################################################################################################
################################################# 아래의 둘 중 하나의 케이스를 사용하여 작업하자. (위의 STEP1,2와 이어서 사용 할것.) #################################################
######################################################### 들어가기 앞서, quality 데이터는 fwver변수만있지, model변수는 없다. #########################################################
############################################################ 그러나, fwver변수가 model변수와 매우 밀접한 관련이 있으므로, ############################################################
########################################################################## 이를 통해 model변수를 만들어준다. #########################################################################
######################################################################################################################################################################################

In [27]:
################################################################### case 를 들어가기 앞서, 실행해야 하는 코드 ###################################################################

In [28]:
print('train quality의 개수: ',len(set(train_quality['user_id']))) # 8281명의 퀄리티 정보
print('test데이터의 펌웨어집합 - train 데이터의 펌웨어 집합:',set(test_quality['fwver'])-set(train_quality['fwver'])) # 그리고 test데이터의 펌웨어버전은 train안에 다 포함됨.

# 따라서, train의 펌웨어를 고려하면, 이를 test에도 고스란히 사용할 수 있다.

train quality의 개수:  8281
test데이터의 펌웨어집합 - train 데이터의 펌웨어 집합: set()


In [29]:
def fw_to_mod(x):

  # 펌웨어를 모델로 바꾸는 함수.

  f,m,e=x.split('.') # front,mid,end
  ch_model= f +'.' + m

  if ch_model == '04.22':
    return 'model_0'
  elif ch_model == '04.16':
    return 'model_1'
  elif ch_model == '04.33':
    return 'model_2'
  elif ch_model == '05.15':
    return 'model_3'
  elif ch_model == '03.11':
    return 'model_4'
  elif ch_model == '04.82':
    return 'model_5'
  elif ch_model == '05.66':
    return 'model_7'
  elif ch_model == '04.73':
    return 'model_8'
  elif ch_model == '09.17':  # 얘는 불확실한 모델명. 판단할수가 없다.
    return 'model_unknwon'

In [30]:
train_quality=train_quality.dropna() # 전체 데이터의 20%에 대한 결측치는 다소 조심스럽지만, 별 수단이 없으므로 제거해준다. (평균으로 하기엔, quality의 정보가 매우 연속적이지 못하다.)
train_quality['model_nm']=train_quality['fwver'].apply(fw_to_mod) # 모델변수 생성 완료

In [31]:
# 더불어,
# tr_qual에 있는 모든 퀄리티들은 사실 float혹은 int형이라 생각하며, 
# object가 나온 이유는 단순히 , 의 존재 때문에 그렇다. 
# 따라서, 모든 값을 float화 시키고, 콤마(,)의 존재를 지운다.

def string2num(x):  
    
    x = str(x).replace(',','') # ,(콤마) 제거
    x = float(x)
    x = int(x) # 소숫점 제거
    return x

for i in range(13): # quality의 개수
   train_quality['quality_'+str(i)]=train_quality['quality_'+str(i)].apply(string2num)

In [32]:
display(train_quality)

# 분석 start 
# 참고) quality에 대한 시간은 다루지 않음.

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,model_nm
0,20201129090000,10000,05.15.2138,0,0,0,0,0,0,0,0,0,0,4,0,0,model_3
1,20201129090000,10000,05.15.2138,0,0,0,0,0,0,0,0,0,0,4,0,0,model_3
2,20201129090000,10000,05.15.2138,0,0,0,0,0,0,0,0,0,0,4,0,0,model_3
3,20201129090000,10000,05.15.2138,0,0,0,0,0,0,0,0,0,0,4,0,0,model_3
4,20201129090000,10000,05.15.2138,0,0,0,0,0,0,0,0,0,0,4,0,0,model_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828619,20201124033000,24997,04.22.1778,0,0,0,0,0,1,0,0,0,0,17,0,0,model_0
828620,20201124033000,24997,04.22.1778,0,0,0,0,0,0,0,0,0,0,17,0,0,model_0
828621,20201124033000,24997,04.22.1778,0,0,0,0,0,3,0,0,0,0,17,0,0,model_0
828622,20201124033000,24997,04.22.1778,0,0,0,0,0,0,0,0,0,0,17,0,0,model_0


In [33]:
#######################################################################################################################################################################################################
####################################################### case2. 전체 모델에 대해 quality를 계산하는 경우 (tmp1과 tmp2에 고스란히 붙여서 사용한다) ######################################################
#######################################################################################################################################################################################################

In [34]:
# model_nm에대해서 더미변수 생성, 나머지 변수 들고감.
dumi=pd.get_dummies(train_quality[['user_id','model_nm','quality_0','quality_1','quality_2','quality_3','quality_4','quality_5','quality_6','quality_7','quality_8','quality_9','quality_10','quality_11','quality_12']])
dumi['model_nm_model_3']=0
dumi['model_nm_model_4']=0
dumi['model_nm_model_6']=0 # 모델3과 모델6은 없기 때문에, 인위적으로 추가

In [35]:
# 마이너스 개수를 세주기 위한 변환함수.
def cnt_minus(x):
  if x==-1:
    return int(1)
  else:
    return int(0)

# 마이너스 이외의 숫자들에 대해서, 평균을 계산할 준비를 위한 변환함수
def not_minus(x):
  if x == -1:
    return np.nan
  else:
    return x

for i in range(13):
  dumi['quality_'+str(i)+'_minus']= dumi['quality_'+str(i)].apply(cnt_minus) # minus는 sum해서 유저내 모델별 count할 예정.
  dumi['quality_'+str(i)+'_rest']= dumi['quality_'+str(i)].apply(not_minus) # 그외 수치는 유저내 모델별 mean할 예정.
  del dumi['quality_'+str(i)]


In [36]:
agg_format={
    'quality_0_minus':sum, 'quality_1_minus':sum, 'quality_2_minus':sum, 'quality_3_minus':sum, 'quality_4_minus':sum, 'quality_5_minus':sum, 'quality_6_minus':sum,  
    'quality_7_minus':sum, 'quality_8_minus':sum, 'quality_9_minus':sum, 'quality_10_minus':sum, 'quality_11_minus':sum, 'quality_12_minus':sum, 
    
    'quality_0_rest':'mean', 'quality_1_rest':'mean', 'quality_2_rest':'mean', 'quality_3_rest':'mean', 'quality_4_rest':'mean', 'quality_5_rest':'mean', 'quality_6_rest':'mean', 
    'quality_7_rest':'mean', 'quality_8_rest':'mean', 'quality_9_rest':'mean', 'quality_10_rest':'mean', 'quality_11_rest':'mean', 'quality_12_rest':'mean', 
            }

# 각 유저 의 전체 모델에 대해서, -1이 발생한 횟수와 평균적인 퀄리티.
dumi = dumi.groupby(['user_id']).agg(agg_format).reset_index()
display(dumi)

,user_id,quality_0_minus,quality_1_minus,quality_2_minus,quality_3_minus,quality_4_minus,quality_5_minus,quality_6_minus,quality_7_minus,quality_8_minus,quality_9_minus,quality_10_minus,quality_11_minus,quality_12_minus,quality_0_rest,quality_1_rest,quality_2_rest,quality_3_rest,quality_4_rest,quality_5_rest,quality_6_rest,quality_7_rest,quality_8_rest,quality_9_rest,quality_10_rest,quality_11_rest,quality_12_rest
0,10000,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,0,0,0.500000,0.000000,0.000000,0.000,0.000,6.000000,0.0,0.0
1,10002,2,2,2,0,0,2,2,0,0,0,0,2,0,0.021277,0.0,0.010638,0,0,0.329787,0.489362,5.750000,0.000,0.125,3.875000,0.0,0.0
2,10004,2,2,2,0,0,2,2,0,0,0,0,2,0,0.000000,0.0,0.000000,0,0,0.181818,3.954545,43.500000,0.000,0.000,2.000000,0.0,0.0
3,10005,10,10,10,0,0,10,10,0,0,0,0,10,0,0.000000,0.0,0.000000,0,0,0.714286,2.571429,18.000000,0.000,0.000,5.000000,0.0,0.0
4,10006,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,0,0,0.333333,0.111111,1.333333,0.000,0.000,4.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7722,24986,21,21,21,0,0,21,21,0,0,0,0,21,0,0.000000,0.2,0.000000,0,0,35.426667,0.093333,0.875000,1.875,0.000,332.125000,0.0,0.0
7723,24992,12,12,12,0,0,12,12,0,0,0,0,12,0,0.000000,0.0,0.000000,0,0,0.791667,0.000000,0.000000,0.000,0.000,6.333333,0.0,0.0
7724,24993,7,7,7,0,0,7,7,0,0,0,0,7,0,0.000000,0.0,0.000000,0,0,0.470588,0.000000,0.000000,0.000,0.000,4.000000,0.0,0.0
7725,24995,10,10,10,0,0,10,10,0,0,0,0,10,0,0.000000,0.0,0.000000,0,0,0.357143,0.000000,0.000000,0.000,0.000,2.500000,0.0,0.0


In [37]:
res_id = list(set(range(10000,train_user_number+10000)) - set(dumi['user_id'])) # 전체 15000명중, quality 데이터셋에서 사용안된 사람들. (7273명)
res_df=pd.DataFrame(np.zeros((7273,27)),columns=dumi.columns ,dtype=int) # 7273명과 27개의 열을 초기화.
res_df['user_id']=res_id # id만 할당.
display(res_df)

,user_id,quality_0_minus,quality_1_minus,quality_2_minus,quality_3_minus,quality_4_minus,quality_5_minus,quality_6_minus,quality_7_minus,quality_8_minus,quality_9_minus,quality_10_minus,quality_11_minus,quality_12_minus,quality_0_rest,quality_1_rest,quality_2_rest,quality_3_rest,quality_4_rest,quality_5_rest,quality_6_rest,quality_7_rest,quality_8_rest,quality_9_rest,quality_10_rest,quality_11_rest,quality_12_rest
0,10001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10003,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,10007,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,10009,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7268,24991,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7269,24994,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7270,24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7271,24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
tmp3= pd.concat([dumi,res_df]).sort_values('user_id').reset_index(drop=True)
display(tmp3)

,user_id,quality_0_minus,quality_1_minus,quality_2_minus,quality_3_minus,quality_4_minus,quality_5_minus,quality_6_minus,quality_7_minus,quality_8_minus,quality_9_minus,quality_10_minus,quality_11_minus,quality_12_minus,quality_0_rest,quality_1_rest,quality_2_rest,quality_3_rest,quality_4_rest,quality_5_rest,quality_6_rest,quality_7_rest,quality_8_rest,quality_9_rest,quality_10_rest,quality_11_rest,quality_12_rest
0,10000,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,0,0,0.500000,0.000000,0.00,0.0,0.000,6.000,0.0,0.0
1,10001,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,0,0,0.000000,0.000000,0.00,0.0,0.000,0.000,0.0,0.0
2,10002,2,2,2,0,0,2,2,0,0,0,0,2,0,0.021277,0.0,0.010638,0,0,0.329787,0.489362,5.75,0.0,0.125,3.875,0.0,0.0
3,10003,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,0,0,0.000000,0.000000,0.00,0.0,0.000,0.000,0.0,0.0
4,10004,2,2,2,0,0,2,2,0,0,0,0,2,0,0.000000,0.0,0.000000,0,0,0.181818,3.954545,43.50,0.0,0.000,2.000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,24995,10,10,10,0,0,10,10,0,0,0,0,10,0,0.000000,0.0,0.000000,0,0,0.357143,0.000000,0.00,0.0,0.000,2.500,0.0,0.0
14996,24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,0,0,0.000000,0.000000,0.00,0.0,0.000,0.000,0.0,0.0
14997,24997,4,4,4,0,0,4,4,0,0,0,0,4,0,0.000000,0.0,0.000000,0,0,0.950000,0.000000,0.00,0.0,0.000,9.500,0.0,0.0
14998,24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,0,0,0.000000,0.000000,0.00,0.0,0.000,0.000,0.0,0.0


In [39]:
################################################# 케이스 선택 끝. 최종적으로 이후 사용할 변수는    ==> tmp3 <==    변수임. #################################################

## 2-4) err_code 변수에 대한 가공

In [40]:
# STEP 4. err_code 변수에 대한 전처리

################################################# 아래의 둘 중 하나의 케이스를 사용하여 작업하자. (위의 STEP1,2,3와 이어서 사용 할것.) #################################################

In [41]:
###############################################################################################################################################################
####################################################### case2. 전체 모델에 대해 errcode를 계산하는 경우  ######################################################
###############################################################################################################################################################

In [42]:
top50=list(train_err['errcode'].value_counts().head(50).index) # 상위 50개만 선정. 이 기준에 대해서는 좀 더 살펴볼 여지가 있다.

# 그리고, errocde를 변환하는 함수를 만들고,
def cvt_errcode(x):
  if x in top50:
    return x
  else:
    return 'etc'

train_err['code_cvt']=train_err['errcode'].apply(cvt_errcode) # 적용.

In [43]:
dumi=pd.get_dummies(train_err[['user_id','model_nm','code_cvt']]) # model_nm과 code_cvt에 대해 더미변수 생성 (user_id 사용이뉴는 dumi변수에 그대로 가져가기 위함.)
tmp4=dumi.groupby(['user_id']).sum().astype(int).reset_index()

def convert_1(x): 
  # 숫자가 있으면 1, 없으면 0으로 바꾸는 함수.
  if x>0:
    return int(1)
  else:
    return int(0)

# model 변수에 대해 16,554,663개가 15000개로 압축되어 count되었기 때문에, 이를 1 또는 0으로 변환.
# tmp1에 아래의 데이터프레임을 저장함.
for i in range(9):
  tmp4['model_nm_model_'+str(i)]=tmp4['model_nm_model_'+str(i)].apply(convert_1)

del train_err['code_cvt']

display(tmp4)

,user_id,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,code_cvt_0,code_cvt_1,code_cvt_100,code_cvt_104,code_cvt_105,code_cvt_13,code_cvt_14,code_cvt_2,code_cvt_21,code_cvt_3,code_cvt_4,code_cvt_5,code_cvt_6,code_cvt_77,code_cvt_78,code_cvt_79,code_cvt_8.0,code_cvt_80,code_cvt_81,code_cvt_82,code_cvt_83,code_cvt_84,code_cvt_85,code_cvt_86,code_cvt_87,code_cvt_88,code_cvt_89,code_cvt_90,code_cvt_91,code_cvt_92,code_cvt_93,code_cvt_94,code_cvt_95,code_cvt_99,code_cvt_B-51049,code_cvt_B-A8002,code_cvt_H-51042,code_cvt_J-30021,code_cvt_NFANDROID2,code_cvt_Q-64002,code_cvt_S-61001,code_cvt_S-65002,code_cvt_UNKNOWN,code_cvt_V-21008,code_cvt_active,code_cvt_connection fail to establish,code_cvt_connection timeout,code_cvt_connectionterminated by local host,code_cvt_etc,code_cvt_standby,code_cvt_terminate by peer user
0,10000,0,0,0,1,0,0,0,0,0,104,212,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10001,0,0,1,0,0,0,0,0,0,183,1274,0,0,0,1,16,7,0,5,18,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,53,0,0,56,0,0,0,0,0,126,0,0,0,0,625,0
2,10002,0,0,0,1,0,0,0,0,0,132,172,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,10003,0,0,1,0,0,0,0,0,0,42,229,0,0,0,0,3,3,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,12,0,7,0,2,0,0
4,10004,1,0,0,0,0,0,0,0,0,98,529,0,0,0,3,9,14,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,1,104,0,0,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,24995,0,0,1,0,0,0,0,0,0,22,125,0,0,0,4,14,4,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,7,0,0,0,0,0,4,0,0,0,0,8,0
14996,24996,0,0,0,1,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14997,24997,1,0,0,0,0,0,0,0,0,146,598,0,0,0,1,2,20,0,13,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,8,0,0,0,0,0,4,0,13,1,0,11,0
14998,24998,1,0,0,0,0,0,0,0,0,13,125,0,0,0,0,1,12,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [44]:
################################################# 케이스 선택 끝. 최종적으로 이후 사용할 변수는    ==> tmp4 <==    변수로 공통적임. #################################################

## 2-5) X_train dataset 생성

In [45]:
# Step5. 위의 4개 데이터(tmp1 ~ tmp4)를 종합하여 1개의 x_train 데이터를 만든다.

In [46]:
# 아래는 Case2에대한 merge이다. Case1에대한 merge는 해결을 못함... 서로의 데이터셋이 충돌하기때문..

In [47]:
display(tmp1)

,user_id,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
0,10000,0,0,0,1,0,0,0,0,0,0,0,8,104,0,1,1,0,0,7,15,16,1,10,59,61,0,0,0,0,0,0,0,0,0,32,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10001,0,0,1,0,0,0,0,0,0,0,0,0,0,53,1,1,0,0,0,10,11,1,15,151,128,0,4,1,3,1,756,751,5,1,22,0,0,0,250,0,10,18,0,1,1,0,0,113,56,1
2,10002,0,0,0,1,0,0,0,0,0,0,0,2,132,1,2,1,0,0,1,13,14,1,4,52,52,0,0,0,0,0,0,0,2,0,25,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10003,0,0,1,0,0,0,0,0,0,0,0,0,0,2,1,1,0,0,0,9,9,0,0,52,30,1,0,0,0,0,28,19,0,0,59,0,0,0,65,0,8,0,0,1,1,2,0,17,1,0
4,10004,1,0,0,0,0,0,0,0,0,0,0,0,1,0,3,4,0,0,0,16,19,3,5,143,91,0,0,0,0,0,140,119,0,0,33,0,0,0,176,0,16,0,0,1,1,0,0,4,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,24995,0,0,1,0,0,0,0,0,0,0,0,0,0,2,5,5,0,0,0,5,10,8,8,26,18,0,0,0,0,0,17,12,0,0,22,0,0,0,31,0,5,0,0,0,0,0,0,9,7,4
14996,24996,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
14997,24997,1,0,0,0,0,0,0,0,0,0,0,0,1,8,1,1,0,0,0,16,18,4,1,181,138,0,0,0,0,0,50,29,0,0,49,0,0,0,223,0,16,17,0,1,1,0,0,58,8,5
14998,24998,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,15,0,1,51,12,0,0,0,0,0,7,1,0,0,4,0,0,0,14,0,15,12,0,1,1,0,0,6,0,0


In [48]:
display(tmp2)

,user_id,cnt,valid_time
0,10000,20,23
1,10001,1452,3
2,10002,17,23
3,10003,24,19
4,10004,102,10
...,...,...,...
14995,24995,30,3
14996,24996,4,0
14997,24997,64,26
14998,24998,37,20


In [49]:
display(tmp3)

,user_id,quality_0_minus,quality_1_minus,quality_2_minus,quality_3_minus,quality_4_minus,quality_5_minus,quality_6_minus,quality_7_minus,quality_8_minus,quality_9_minus,quality_10_minus,quality_11_minus,quality_12_minus,quality_0_rest,quality_1_rest,quality_2_rest,quality_3_rest,quality_4_rest,quality_5_rest,quality_6_rest,quality_7_rest,quality_8_rest,quality_9_rest,quality_10_rest,quality_11_rest,quality_12_rest
0,10000,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,0,0,0.500000,0.000000,0.00,0.0,0.000,6.000,0.0,0.0
1,10001,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,0,0,0.000000,0.000000,0.00,0.0,0.000,0.000,0.0,0.0
2,10002,2,2,2,0,0,2,2,0,0,0,0,2,0,0.021277,0.0,0.010638,0,0,0.329787,0.489362,5.75,0.0,0.125,3.875,0.0,0.0
3,10003,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,0,0,0.000000,0.000000,0.00,0.0,0.000,0.000,0.0,0.0
4,10004,2,2,2,0,0,2,2,0,0,0,0,2,0,0.000000,0.0,0.000000,0,0,0.181818,3.954545,43.50,0.0,0.000,2.000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,24995,10,10,10,0,0,10,10,0,0,0,0,10,0,0.000000,0.0,0.000000,0,0,0.357143,0.000000,0.00,0.0,0.000,2.500,0.0,0.0
14996,24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,0,0,0.000000,0.000000,0.00,0.0,0.000,0.000,0.0,0.0
14997,24997,4,4,4,0,0,4,4,0,0,0,0,4,0,0.000000,0.0,0.000000,0,0,0.950000,0.000000,0.00,0.0,0.000,9.500,0.0,0.0
14998,24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,0,0,0.000000,0.000000,0.00,0.0,0.000,0.000,0.0,0.0


In [50]:
display(tmp4)

,user_id,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,code_cvt_0,code_cvt_1,code_cvt_100,code_cvt_104,code_cvt_105,code_cvt_13,code_cvt_14,code_cvt_2,code_cvt_21,code_cvt_3,code_cvt_4,code_cvt_5,code_cvt_6,code_cvt_77,code_cvt_78,code_cvt_79,code_cvt_8.0,code_cvt_80,code_cvt_81,code_cvt_82,code_cvt_83,code_cvt_84,code_cvt_85,code_cvt_86,code_cvt_87,code_cvt_88,code_cvt_89,code_cvt_90,code_cvt_91,code_cvt_92,code_cvt_93,code_cvt_94,code_cvt_95,code_cvt_99,code_cvt_B-51049,code_cvt_B-A8002,code_cvt_H-51042,code_cvt_J-30021,code_cvt_NFANDROID2,code_cvt_Q-64002,code_cvt_S-61001,code_cvt_S-65002,code_cvt_UNKNOWN,code_cvt_V-21008,code_cvt_active,code_cvt_connection fail to establish,code_cvt_connection timeout,code_cvt_connectionterminated by local host,code_cvt_etc,code_cvt_standby,code_cvt_terminate by peer user
0,10000,0,0,0,1,0,0,0,0,0,104,212,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10001,0,0,1,0,0,0,0,0,0,183,1274,0,0,0,1,16,7,0,5,18,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,53,0,0,56,0,0,0,0,0,126,0,0,0,0,625,0
2,10002,0,0,0,1,0,0,0,0,0,132,172,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,10003,0,0,1,0,0,0,0,0,0,42,229,0,0,0,0,3,3,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,12,0,7,0,2,0,0
4,10004,1,0,0,0,0,0,0,0,0,98,529,0,0,0,3,9,14,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,1,104,0,0,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,24995,0,0,1,0,0,0,0,0,0,22,125,0,0,0,4,14,4,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,7,0,0,0,0,0,4,0,0,0,0,8,0
14996,24996,0,0,0,1,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14997,24997,1,0,0,0,0,0,0,0,0,146,598,0,0,0,1,2,20,0,13,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,8,0,0,0,0,0,4,0,13,1,0,11,0
14998,24998,1,0,0,0,0,0,0,0,0,13,125,0,0,0,0,1,12,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [51]:
X = pd.merge(tmp1,tmp2,on=['user_id'],how='left')
X = pd.merge(X,tmp3,on=['user_id'],how='left')
X = pd.merge(X,tmp4,on=['user_id','model_nm_model_0','model_nm_model_1','model_nm_model_2','model_nm_model_3','model_nm_model_4','model_nm_model_5','model_nm_model_6','model_nm_model_7','model_nm_model_8'],how='left')

In [52]:
display(X)

,user_id,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,...,code_cvt_5,code_cvt_6,code_cvt_77,code_cvt_78,code_cvt_79,code_cvt_8.0,code_cvt_80,code_cvt_81,code_cvt_82,code_cvt_83,code_cvt_84,code_cvt_85,code_cvt_86,code_cvt_87,code_cvt_88,code_cvt_89,code_cvt_90,code_cvt_91,code_cvt_92,code_cvt_93,code_cvt_94,code_cvt_95,code_cvt_99,code_cvt_B-51049,code_cvt_B-A8002,code_cvt_H-51042,code_cvt_J-30021,code_cvt_NFANDROID2,code_cvt_Q-64002,code_cvt_S-61001,code_cvt_S-65002,code_cvt_UNKNOWN,code_cvt_V-21008,code_cvt_active,code_cvt_connection fail to establish,code_cvt_connection timeout,code_cvt_connectionterminated by local host,code_cvt_etc,code_cvt_standby,code_cvt_terminate by peer user
0,10000,0,0,0,1,0,0,0,0,0,0,0,8,104,0,1,1,0,0,7,15,16,1,10,59,61,0,0,0,0,0,0,0,0,0,32,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10001,0,0,1,0,0,0,0,0,0,0,0,0,0,53,1,1,0,0,0,10,11,1,15,151,128,0,4,1,3,1,756,751,5,1,22,0,0,0,250,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,53,0,0,56,0,0,0,0,0,126,0,0,0,0,625,0
2,10002,0,0,0,1,0,0,0,0,0,0,0,2,132,1,2,1,0,0,1,13,14,1,4,52,52,0,0,0,0,0,0,0,2,0,25,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,10003,0,0,1,0,0,0,0,0,0,0,0,0,0,2,1,1,0,0,0,9,9,0,0,52,30,1,0,0,0,0,28,19,0,0,59,0,0,0,65,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,12,0,7,0,2,0,0
4,10004,1,0,0,0,0,0,0,0,0,0,0,0,1,0,3,4,0,0,0,16,19,3,5,143,91,0,0,0,0,0,140,119,0,0,33,0,0,0,176,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,1,104,0,0,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,24995,0,0,1,0,0,0,0,0,0,0,0,0,0,2,5,5,0,0,0,5,10,8,8,26,18,0,0,0,0,0,17,12,0,0,22,0,0,0,31,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,7,0,0,0,0,0,4,0,0,0,0,8,0
14996,24996,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14997,24997,1,0,0,0,0,0,0,0,0,0,0,0,1,8,1,1,0,0,0,16,18,4,1,181,138,0,0,0,0,0,50,29,0,0,49,0,0,0,223,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,8,0,0,0,0,0,4,0,13,1,0,11,0
14998,24998,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,15,0,1,51,12,0,0,0,0,0,7,1,0,0,4,0,0,0,14,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [53]:
# X저장
del X['user_id']
np.savetxt(os.path.join(current_day,'x_train.csv'),X)
# pandas 저장
X.to_csv(os.path.join(current_day,'x_train_df.csv'),index=False)

## 2-6) train_problem.csv 파일에 대한 가공 (y변수 만들기)

In [54]:
# Step6. target변수에 대해서 가공. 0과 1이지만, 보내는 횟수 자체를 그룹화해서 살펴본다.
# 이때, 너무나 극심한 불균형이므로, count가 3이상인 그룹은 하나로 묶는다.

In [55]:
def le_three(x):
  x=int(x)
  if x >=3 :
    return int(3)
  else:
    return int(x)

In [56]:
y_tmp = train_problem['user_id'].value_counts().reset_index().rename(columns={'index':'user_id','user_id':'count'}) # 각 유저별, 문의를 한 횟수.
y_tmp['count']=y_tmp['count'].apply(le_three)
display(y_tmp)

,user_id,count
0,24407,3
1,11016,3
2,20636,3
3,11528,3
4,24138,3
...,...,...
4995,19613,1
4996,19739,1
4997,13596,1
4998,15645,1


In [57]:
y=np.zeros(15000) # 초기화.

In [58]:
for i in y_tmp.values: # 문의를 보내지 않은 사람들까지 고려한 y변수 생성.
  y[i[0]-10000] =i[1] 

In [59]:
train_y=y.astype('int')
print(train_y.shape)
display(train_y) # nd_array

(15000,)


array([0, 1, 0, ..., 1, 1, 0])

In [60]:
# y저장
np.savetxt(os.path.join(current_day,'y_train.csv'),train_y)

# 3) Test 전처리

In [61]:
# 바쁘므로, Case2에 대한 전처리만 진행함.
# 들어가기 앞서, test_err 데이터셋에는 id 43262번에 대한 정보가 없다. 
# 이를 주의하며, 각 전처리에서 43262번에 대한 정보를 추가해주도록 하자.

In [62]:
display(test_err[test_err['user_id']==43262]) # 누락확인.

,user_id,time,model_nm,fwver,errtype,errcode


## 2-1) model(fwver)변수와 err_type 변수 가공

In [63]:
# STEP 1. model(fwver)과 err_type 변수 사용

################################################# 아래의 둘 중 하나의 케이스를 사용하여 작업하자. #################################################

In [64]:
#######################################################################################################################################################################################################
########################## case2. 각 유저에 대해 에러가 발생한 횟수를 단순히 합한 경우: y의 카운트가 문제가 되지는 않지만, 어느 모델에서 문제가 생겼는지 혼란스러울 수 있다. ##########################
#######################################################################################################################################################################################################

In [65]:
test_err['errtype']=test_err.errtype.astype('object') # 더미변수 생성하기 위해 카테고리화
dumi=pd.get_dummies(test_err[['user_id','model_nm','errtype']]) # model_nm과 errtype에 대해 더미변수 생성 (user_id 사용이뉴는 dumi변수에 그대로 가져가기 위함.)
tmp1=dumi.groupby(['user_id']).sum().astype(int).reset_index()

def convert_1(x): 
  # 숫자가 있으면 1, 없으면 0으로 바꾸는 함수.
  if x>0:
    return int(1)
  else:
    return int(0)

# model 변수에 대해 16,554,663개가 15000개로 압축되어 count되었기 때문에, 이를 1 또는 0으로 변환.
# tmp1에 아래의 데이터프레임을 저장함.
for i in range(9):
  tmp1['model_nm_model_'+str(i)]=tmp1['model_nm_model_'+str(i)].apply(convert_1)

display(tmp1)

,user_id,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
0,30000,0,1,1,0,0,0,0,0,0,0,0,0,0,62,1,1,0,0,0,16,16,0,3,67,37,9,3,0,1,0,983,1308,5,2,32,0,0,0,83,2,16,0,1,3,3,0,2,92,0,2
1,30001,0,0,0,1,0,0,0,0,0,0,0,3,98,3,4,4,0,0,0,12,16,4,6,53,55,0,0,0,0,0,0,0,0,0,24,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,30002,1,0,0,0,0,0,0,0,0,0,0,0,0,79,2,1,0,0,0,16,16,0,1,173,133,0,0,0,0,0,60,40,2,0,40,0,0,0,230,0,16,0,0,1,1,0,14,113,1,2
3,30003,1,0,0,0,0,0,0,0,0,0,0,0,0,107,2,2,0,0,0,4,5,1,7,8,12,0,1,0,1,0,3,2,0,0,5,2,2,0,134,0,4,16,3,1,1,0,0,40,8,0
4,30004,0,1,1,0,0,0,0,0,0,0,0,0,2,6,1,1,0,0,0,10,12,1,18,80,67,6,0,0,0,0,10,8,0,0,14,1,1,0,332,152,11,6,1,1,1,1,0,48,90,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14993,44994,0,1,0,0,0,0,0,0,0,0,0,0,2,12,5,5,0,0,0,16,19,3,9,123,93,12,0,0,0,0,197,204,0,0,31,0,0,0,179,2,16,0,0,0,0,0,0,100,87,0
14994,44995,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,14,15,1,4,149,98,0,0,0,0,0,0,0,0,0,1,0,0,0,183,0,14,1,0,1,1,0,0,12,17,0
14995,44996,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,12,13,1,3,129,90,2,0,0,0,0,138,165,0,0,48,0,0,0,163,0,12,1118,0,1,1,1,0,329,2,3
14996,44997,1,0,0,0,0,0,0,0,0,0,0,0,1,4,2,2,0,0,0,15,16,0,1,73,44,0,1,0,1,0,11127,13161,0,3,44,0,0,0,78,0,14,0,0,1,1,3,0,30,49,0


In [66]:
# 43262번 처리 잊지말자.

id_43262 = pd.DataFrame(np.array([['43262']+[0]*50],dtype=int),columns=tmp1.columns) 
tmp1=tmp1.append(id_43262,ignore_index=True)
tmp1=tmp1.sort_values('user_id')
display(tmp1)

,user_id,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
0,30000,0,1,1,0,0,0,0,0,0,0,0,0,0,62,1,1,0,0,0,16,16,0,3,67,37,9,3,0,1,0,983,1308,5,2,32,0,0,0,83,2,16,0,1,3,3,0,2,92,0,2
1,30001,0,0,0,1,0,0,0,0,0,0,0,3,98,3,4,4,0,0,0,12,16,4,6,53,55,0,0,0,0,0,0,0,0,0,24,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,30002,1,0,0,0,0,0,0,0,0,0,0,0,0,79,2,1,0,0,0,16,16,0,1,173,133,0,0,0,0,0,60,40,2,0,40,0,0,0,230,0,16,0,0,1,1,0,14,113,1,2
3,30003,1,0,0,0,0,0,0,0,0,0,0,0,0,107,2,2,0,0,0,4,5,1,7,8,12,0,1,0,1,0,3,2,0,0,5,2,2,0,134,0,4,16,3,1,1,0,0,40,8,0
4,30004,0,1,1,0,0,0,0,0,0,0,0,0,2,6,1,1,0,0,0,10,12,1,18,80,67,6,0,0,0,0,10,8,0,0,14,1,1,0,332,152,11,6,1,1,1,1,0,48,90,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14993,44994,0,1,0,0,0,0,0,0,0,0,0,0,2,12,5,5,0,0,0,16,19,3,9,123,93,12,0,0,0,0,197,204,0,0,31,0,0,0,179,2,16,0,0,0,0,0,0,100,87,0
14994,44995,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,14,15,1,4,149,98,0,0,0,0,0,0,0,0,0,1,0,0,0,183,0,14,1,0,1,1,0,0,12,17,0
14995,44996,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,12,13,1,3,129,90,2,0,0,0,0,138,165,0,0,48,0,0,0,163,0,12,1118,0,1,1,1,0,329,2,3
14996,44997,1,0,0,0,0,0,0,0,0,0,0,0,1,4,2,2,0,0,0,15,16,0,1,73,44,0,1,0,1,0,11127,13161,0,3,44,0,0,0,78,0,14,0,0,1,1,3,0,30,49,0


In [67]:
################################################# 케이스 선택 끝. 최종적으로 이후 사용할 변수는    ==> tmp1 <==    변수임. #################################################

## 2-2) 가장 많은 error가 발생한 시점과, 그때의 error 발생 건수 가공

In [68]:
# STEP 2. 가장 많은 error가 발생한 시점과, 그때의 error 발생 건수.

################################################# 아래의 둘 중 하나의 케이스를 사용하여 작업하자. (위의 STEP1과 이어서 사용 할것.) #################################################

In [69]:
#######################################################################################################################################################################################################
########################## case2. 각 유저에 대해 에러가 발생한 횟수를 단순히 합한 경우: y의 카운트가 문제가 되지는 않지만, 어느 모델에서 문제가 생겼는지 혼란스러울 수 있다. ##########################
#######################################################################################################################################################################################################

In [70]:
# 시간 변환함수. (출처: 데이콘 baseline)
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    #hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year,month, day)

In [71]:
test_err['month_day']=test_err['time'].apply(make_datetime) # 우리는 월,일만 살펴본다.
test_err['cnt']=1 # 카운트를 세기위한 보조변수

In [72]:
tmp2= test_err[['user_id','month_day','cnt']].groupby(['user_id','month_day']).count()

del test_err['month_day'], test_err['cnt'] # 만들었던거 삭제

# user_id별 에러가 최대로 발생한 날과, 그 에러의 수. (모델과 관련 없이 에러를 카운트 하는 case2와 관련이 깊다.)
tmp2= tmp2.sort_values(by='cnt', ascending=False).reset_index().groupby(['user_id']).head(1).sort_values('user_id').reset_index(drop=True)
display(tmp2)

,user_id,month_day,cnt
0,30000,2020-11-07,808
1,30001,2020-11-07,22
2,30002,2020-11-26,67
3,30003,2020-11-20,75
4,30004,2020-11-19,85
...,...,...,...
14993,44994,2020-11-15,90
14994,44995,2020-11-06,35
14995,44996,2020-11-15,430
14996,44997,2020-11-26,2606


In [73]:
###################################################################### 케이스 선택 끝. ######################################################################

In [74]:
#########################################################################################################################################
####################################### 위의 케이스 둘 중 하나를 선택했으면, 아래에서 추가가공을 진행 #######################################
#########################################################################################################################################

In [75]:
################################################################# 이것 까지 해야 완료. #################################################################


max_time = 20201130235959 # 최대시간을 11월 30일 23시 59분 59초라 설정해보자. (우리는 빈도로 접근하기 때문에, 최대값으로 12월이 넘어가는 애러발생시점이 나오지 않는다.)
tmp2['due']=make_datetime(max_time)
tmp2['valid_time']=(tmp2['due']-tmp2['month_day']).dt.days # 시간변수 추가.

del tmp2['month_day'], tmp2['due']
display(tmp2) # cnt변수는 에러가 최대로 발생한 시점의 에러 개수 // valid_time 변수는 에러가 최대로 발생한 시점으로부터 문의를 보낼수 있는 유효시간

,user_id,cnt,valid_time
0,30000,808,23
1,30001,22,23
2,30002,67,4
3,30003,75,10
4,30004,85,11
...,...,...,...
14993,44994,90,15
14994,44995,35,24
14995,44996,430,15
14996,44997,2606,4


In [76]:
# 43262번 처리 잊지말자.

id_43262 = pd.DataFrame(np.array([['43262',0,0]],dtype=int),columns=tmp2.columns)  # cnt는 0번, valid time도 0으로 치환하였다. (error가 하나도 안떳기 때문에, 메세지를 보낼 가능성을 최대한 줄였다.)
tmp2=tmp2.append(id_43262,ignore_index=True)
tmp2=tmp2.sort_values('user_id')
display(tmp2)

,user_id,cnt,valid_time
0,30000,808,23
1,30001,22,23
2,30002,67,4
3,30003,75,10
4,30004,85,11
...,...,...,...
14993,44994,90,15
14994,44995,35,24
14995,44996,430,15
14996,44997,2606,4


In [77]:
################################################# 여기까지 하고 나서 나온 최종적인 변수는    ==> tmp2 <==    를 사용함. #################################################

## 2-3) quality 변수에 대한 가공

In [78]:
# STEP 3. err_code 변수에 대한 전처리


######################################################################################################################################################################################
################################################# 아래의 둘 중 하나의 케이스를 사용하여 작업하자. (위의 STEP1,2와 이어서 사용 할것.) #################################################
######################################################### 들어가기 앞서, quality 데이터는 fwver변수만있지, model변수는 없다. #########################################################
############################################################ 그러나, fwver변수가 model변수와 매우 밀접한 관련이 있으므로, ############################################################
########################################################################## 이를 통해 model변수를 만들어준다. #########################################################################
######################################################################################################################################################################################

In [79]:
################################################################### case 를 들어가기 앞서, 실행해야 하는 코드 ###################################################################

In [80]:
def fw_to_mod(x):

  # 펌웨어를 모델로 바꾸는 함수.

  f,m,e=x.split('.') # front,mid,end
  ch_model= f +'.' + m

  if ch_model == '04.22':
    return 'model_0'
  elif ch_model == '04.16':
    return 'model_1'
  elif ch_model == '04.33':
    return 'model_2'
  elif ch_model == '05.15':
    return 'model_3'
  elif ch_model == '03.11':
    return 'model_4'
  elif ch_model == '04.82':
    return 'model_5'
  elif ch_model == '05.66':
    return 'model_7'
  elif ch_model == '04.73':
    return 'model_8'
  elif ch_model == '09.17':  # 얘는 불확실한 모델명. 판단할수가 없다.
    return 'model_unknwon'

In [81]:
# 여기 drop하는 과정에서, 43262번이 누락된다. 주의할것.

test_quality=test_quality.dropna() # 전체 데이터의 20%에 대한 결측치는 다소 조심스럽지만, 별 수단이 없으므로 제거해준다. (평균으로 하기엔, quality의 정보가 매우 연속적이지 못하다.)
test_quality['model_nm']=test_quality['fwver'].apply(fw_to_mod) # 모델변수 생성 완료

In [82]:
# 더불어,
# tr_qual에 있는 모든 퀄리티들은 사실 float혹은 int형이라 생각하며, 
# object가 나온 이유는 단순히 , 의 존재 때문에 그렇다. 
# 따라서, 모든 값을 float화 시키고, 콤마(,)의 존재를 지운다.

def string2num(x):  
    
    x = str(x).replace(',','') # ,(콤마) 제거
    x = float(x)
    x = int(x) # 소숫점 제거
    return x

for i in range(13): # quality의 개수
   test_quality['quality_'+str(i)]=test_quality['quality_'+str(i)].apply(string2num)

In [83]:
display(test_quality)

# 분석 start 
# 참고) quality에 대한 시간은 다루지 않음.

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,model_nm
0,20201128195000,30000,04.33.1261,0,0,0,0,0,0,0,5,0,0,2,0,0,model_2
1,20201128195000,30000,04.33.1261,0,0,0,0,0,0,0,5,0,0,2,0,0,model_2
2,20201128195000,30000,04.33.1261,0,0,0,0,0,0,0,5,0,0,2,0,0,model_2
3,20201128195000,30000,04.33.1261,0,0,0,0,0,0,0,5,0,0,2,0,0,model_2
4,20201128195000,30000,04.33.1261,0,0,0,0,0,0,0,5,0,0,2,0,0,model_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747967,20201125180000,44997,04.22.1750,0,0,0,0,0,2,0,0,0,0,3,0,0,model_0
747968,20201125180000,44997,04.22.1750,0,0,0,0,0,0,0,0,0,0,3,0,0,model_0
747969,20201125180000,44997,04.22.1750,0,0,0,0,0,0,0,0,0,0,3,0,0,model_0
747970,20201125180000,44997,04.22.1750,0,0,0,0,0,0,0,0,0,0,3,0,0,model_0


In [84]:
#######################################################################################################################################################################################################
####################################################### case2. 전체 모델에 대해 quality를 계산하는 경우 (tmp1과 tmp2에 고스란히 붙여서 사용한다) ######################################################
#######################################################################################################################################################################################################

In [85]:
# model_nm에대해서 더미변수 생성, 나머지 변수 들고감.
dumi=pd.get_dummies(test_quality[['user_id','model_nm','quality_0','quality_1','quality_2','quality_3','quality_4','quality_5','quality_6','quality_7','quality_8','quality_9','quality_10','quality_11','quality_12']])
dumi['model_nm_model_3']=0
dumi['model_nm_model_4']=0
dumi['model_nm_model_6']=0 # 모델3과 모델6은 없기 때문에, 인위적으로 추가

In [86]:
# 마이너스 개수를 세주기 위한 변환함수.
def cnt_minus(x):
  if x==-1:
    return int(1)
  else:
    return int(0)

# 마이너스 이외의 숫자들에 대해서, 평균을 계산할 준비를 위한 변환함수
def not_minus(x):
  if x == -1:
    return np.nan
  else:
    return x

for i in range(13):
  dumi['quality_'+str(i)+'_minus']= dumi['quality_'+str(i)].apply(cnt_minus) # minus는 sum해서 유저내 모델별 count할 예정.
  dumi['quality_'+str(i)+'_rest']= dumi['quality_'+str(i)].apply(not_minus) # 그외 수치는 유저내 모델별 mean할 예정.
  del dumi['quality_'+str(i)]


In [87]:
agg_format={
    'quality_0_minus':sum, 'quality_1_minus':sum, 'quality_2_minus':sum, 'quality_3_minus':sum, 'quality_4_minus':sum, 'quality_5_minus':sum, 'quality_6_minus':sum,  
    'quality_7_minus':sum, 'quality_8_minus':sum, 'quality_9_minus':sum, 'quality_10_minus':sum, 'quality_11_minus':sum, 'quality_12_minus':sum, 
    
    'quality_0_rest':'mean', 'quality_1_rest':'mean', 'quality_2_rest':'mean', 'quality_3_rest':'mean', 'quality_4_rest':'mean', 'quality_5_rest':'mean', 'quality_6_rest':'mean', 
    'quality_7_rest':'mean', 'quality_8_rest':'mean', 'quality_9_rest':'mean', 'quality_10_rest':'mean', 'quality_11_rest':'mean', 'quality_12_rest':'mean', 
            }

# 각 유저 의 전체 모델에 대해서, -1이 발생한 횟수와 평균적인 퀄리티.
dumi = dumi.groupby(['user_id']).agg(agg_format).reset_index()
display(dumi)

,user_id,quality_0_minus,quality_1_minus,quality_2_minus,quality_3_minus,quality_4_minus,quality_5_minus,quality_6_minus,quality_7_minus,quality_8_minus,quality_9_minus,quality_10_minus,quality_11_minus,quality_12_minus,quality_0_rest,quality_1_rest,quality_2_rest,quality_3_rest,quality_4_rest,quality_5_rest,quality_6_rest,quality_7_rest,quality_8_rest,quality_9_rest,quality_10_rest,quality_11_rest,quality_12_rest
0,30000,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,0,0,0.166667,0.416667,5.00,0.0,0.00,2.000000,0.0,0.0
1,30001,9,9,9,0,0,9,9,0,0,0,0,9,0,0.000000,0.0,0.000000,0,0,94.925926,0.000000,0.00,0.0,0.00,854.333333,0.0,0.0
2,30002,15,15,15,0,0,15,15,0,0,0,0,15,0,0.000000,0.0,0.000000,0,0,1.266667,0.000000,0.00,0.0,0.00,13.300000,0.0,0.0
3,30003,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,0,0,0.200000,3.666667,44.00,0.0,0.00,2.400000,0.0,0.0
4,30004,6,6,6,0,0,6,6,0,0,0,0,6,0,0.000000,0.0,0.000000,0,0,0.111111,1.833333,16.50,0.0,0.00,1.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7712,44990,4,4,4,0,0,4,4,0,0,0,0,4,0,0.000000,0.0,0.000000,0,0,1.562500,0.000000,0.00,0.0,0.00,16.666667,0.0,0.0
7713,44993,2,2,2,0,0,2,2,0,0,0,0,2,0,0.195652,0.0,0.195652,0,0,0.304348,2.369565,27.25,0.0,2.25,3.500000,0.0,0.0
7714,44994,11,11,11,0,0,11,11,0,0,0,0,11,0,0.000000,0.0,0.000000,0,0,2.000000,0.000000,0.00,0.0,0.00,2.000000,0.0,0.0
7715,44996,5,5,5,0,0,5,5,0,0,0,0,5,0,0.000000,0.0,0.000000,0,0,0.359223,0.000000,0.00,0.0,0.00,4.111111,0.0,0.0


In [88]:
res_id = list(set(range(30000,test_user_number+30000)) - set(dumi['user_id'])) # 전체 15000명중, quality 데이터셋에서 사용안된 사람들. (7282명)
res_df=pd.DataFrame(np.zeros((7282,27)),columns=dumi.columns ,dtype=int) # 7282명과 27개의 열을 초기화.
res_df['user_id']=res_id # id만 할당.
display(res_df)

,user_id,quality_0_minus,quality_1_minus,quality_2_minus,quality_3_minus,quality_4_minus,quality_5_minus,quality_6_minus,quality_7_minus,quality_8_minus,quality_9_minus,quality_10_minus,quality_11_minus,quality_12_minus,quality_0_rest,quality_1_rest,quality_2_rest,quality_3_rest,quality_4_rest,quality_5_rest,quality_6_rest,quality_7_rest,quality_8_rest,quality_9_rest,quality_10_rest,quality_11_rest,quality_12_rest
0,32768,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,32773,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,32776,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,32780,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,32782,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7277,32759,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7278,32760,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7279,32762,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7280,32763,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [89]:
tmp3= pd.concat([dumi,res_df]).sort_values('user_id').reset_index(drop=True)
display(tmp3)

,user_id,quality_0_minus,quality_1_minus,quality_2_minus,quality_3_minus,quality_4_minus,quality_5_minus,quality_6_minus,quality_7_minus,quality_8_minus,quality_9_minus,quality_10_minus,quality_11_minus,quality_12_minus,quality_0_rest,quality_1_rest,quality_2_rest,quality_3_rest,quality_4_rest,quality_5_rest,quality_6_rest,quality_7_rest,quality_8_rest,quality_9_rest,quality_10_rest,quality_11_rest,quality_12_rest
0,30000,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.166667,0.416667,5.0,0.0,0.0,2.000000,0.0,0.0
1,30001,9,9,9,0,0,9,9,0,0,0,0,9,0,0.0,0.0,0.0,0,0,94.925926,0.000000,0.0,0.0,0.0,854.333333,0.0,0.0
2,30002,15,15,15,0,0,15,15,0,0,0,0,15,0,0.0,0.0,0.0,0,0,1.266667,0.000000,0.0,0.0,0.0,13.300000,0.0,0.0
3,30003,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.200000,3.666667,44.0,0.0,0.0,2.400000,0.0,0.0
4,30004,6,6,6,0,0,6,6,0,0,0,0,6,0,0.0,0.0,0.0,0,0,0.111111,1.833333,16.5,0.0,0.0,1.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,44994,11,11,11,0,0,11,11,0,0,0,0,11,0,0.0,0.0,0.0,0,0,2.000000,0.000000,0.0,0.0,0.0,2.000000,0.0,0.0
14995,44995,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
14996,44996,5,5,5,0,0,5,5,0,0,0,0,5,0,0.0,0.0,0.0,0,0,0.359223,0.000000,0.0,0.0,0.0,4.111111,0.0,0.0
14997,44997,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.541667,0.000000,0.0,0.0,0.0,6.500000,0.0,0.0


In [90]:
# 43262번에 대해 확인하고 간다.
# 주의해야할 점은, 43262번은 결측치가 존재하여, quality변수가 있음에도 누락되었다.
# 이를 보완하는 작업을 거쳐준다.

test_quality=pd.read_csv(PATH+'test_quality_data.csv') # 다시 불러서, 추가시켜줌.
display(tmp3[tmp3['user_id']==43262]) 

,user_id,quality_0_minus,quality_1_minus,quality_2_minus,quality_3_minus,quality_4_minus,quality_5_minus,quality_6_minus,quality_7_minus,quality_8_minus,quality_9_minus,quality_10_minus,quality_11_minus,quality_12_minus,quality_0_rest,quality_1_rest,quality_2_rest,quality_3_rest,quality_4_rest,quality_5_rest,quality_6_rest,quality_7_rest,quality_8_rest,quality_9_rest,quality_10_rest,quality_11_rest,quality_12_rest
13262,43262,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
# 참고)
# 43262번의 정보. quality_0번과 quality_2번이 전부 없다.
display(test_quality[test_quality['user_id']==43262].info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 444 entries, 659616 to 660059
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   time        444 non-null    int64  
 1   user_id     444 non-null    int64  
 2   fwver       0 non-null      object 
 3   quality_0   0 non-null      float64
 4   quality_1   444 non-null    object 
 5   quality_2   0 non-null      float64
 6   quality_3   444 non-null    int64  
 7   quality_4   444 non-null    int64  
 8   quality_5   444 non-null    object 
 9   quality_6   444 non-null    int64  
 10  quality_7   444 non-null    object 
 11  quality_8   444 non-null    object 
 12  quality_9   444 non-null    object 
 13  quality_10  444 non-null    object 
 14  quality_11  444 non-null    int64  
 15  quality_12  444 non-null    int64  
dtypes: float64(2), int64(7), object(7)
memory usage: 59.0+ KB


None

In [92]:
# 값을 살리기 위해, quality0과 2에 대해서 0으로 결측치를 대체하고 간다. (어차피 애러가 로그에 적히질 않았으니 무방하다고 판단.)
id_43262=test_quality[test_quality['user_id']==43262].fillna(0) 

In [93]:
# 똑같이 절차를 밟아준다.

for i in range(13): # quality의 개수
   id_43262['quality_'+str(i)]=id_43262['quality_'+str(i)].apply(string2num)

for i in range(13):
  id_43262['quality_'+str(i)+'_minus']= id_43262['quality_'+str(i)].apply(cnt_minus) # minus는 sum해서 유저내 모델별 count할 예정.
  id_43262['quality_'+str(i)+'_rest']= id_43262['quality_'+str(i)].apply(not_minus) # 그외 수치는 유저내 모델별 mean할 예정.
  del id_43262['quality_'+str(i)]


In [94]:
del id_43262['fwver']

agg_format={
    'quality_0_minus':sum, 'quality_1_minus':sum, 'quality_2_minus':sum, 'quality_3_minus':sum, 'quality_4_minus':sum, 'quality_5_minus':sum, 'quality_6_minus':sum,  
    'quality_7_minus':sum, 'quality_8_minus':sum, 'quality_9_minus':sum, 'quality_10_minus':sum, 'quality_11_minus':sum, 'quality_12_minus':sum, 
    
    'quality_0_rest':'mean', 'quality_1_rest':'mean', 'quality_2_rest':'mean', 'quality_3_rest':'mean', 'quality_4_rest':'mean', 'quality_5_rest':'mean', 'quality_6_rest':'mean', 
    'quality_7_rest':'mean', 'quality_8_rest':'mean', 'quality_9_rest':'mean', 'quality_10_rest':'mean', 'quality_11_rest':'mean', 'quality_12_rest':'mean', 
            }

# 각 유저 의 전체 모델에 대해서, -1이 발생한 횟수와 평균적인 퀄리티.
id_43262 = id_43262.groupby('user_id').agg(agg_format).reset_index()
display(id_43262)

,user_id,quality_0_minus,quality_1_minus,quality_2_minus,quality_3_minus,quality_4_minus,quality_5_minus,quality_6_minus,quality_7_minus,quality_8_minus,quality_9_minus,quality_10_minus,quality_11_minus,quality_12_minus,quality_0_rest,quality_1_rest,quality_2_rest,quality_3_rest,quality_4_rest,quality_5_rest,quality_6_rest,quality_7_rest,quality_8_rest,quality_9_rest,quality_10_rest,quality_11_rest,quality_12_rest
0,43262,0,162,0,0,0,160,163,0,0,0,0,162,0,0,0.0,0,0,0,2.232394,0.060498,0.459459,0,0,17.135135,0.0,0


In [95]:
tmp3=tmp3.drop(43262-30000,axis=0) # 빼주고,
tmp3=tmp3.append(id_43262,ignore_index=True)
tmp3=tmp3.sort_values('user_id')
display(tmp3)

,user_id,quality_0_minus,quality_1_minus,quality_2_minus,quality_3_minus,quality_4_minus,quality_5_minus,quality_6_minus,quality_7_minus,quality_8_minus,quality_9_minus,quality_10_minus,quality_11_minus,quality_12_minus,quality_0_rest,quality_1_rest,quality_2_rest,quality_3_rest,quality_4_rest,quality_5_rest,quality_6_rest,quality_7_rest,quality_8_rest,quality_9_rest,quality_10_rest,quality_11_rest,quality_12_rest
0,30000,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.166667,0.416667,5.0,0.0,0.0,2.000000,0.0,0.0
1,30001,9,9,9,0,0,9,9,0,0,0,0,9,0,0.0,0.0,0.0,0,0,94.925926,0.000000,0.0,0.0,0.0,854.333333,0.0,0.0
2,30002,15,15,15,0,0,15,15,0,0,0,0,15,0,0.0,0.0,0.0,0,0,1.266667,0.000000,0.0,0.0,0.0,13.300000,0.0,0.0
3,30003,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.200000,3.666667,44.0,0.0,0.0,2.400000,0.0,0.0
4,30004,6,6,6,0,0,6,6,0,0,0,0,6,0,0.0,0.0,0.0,0,0,0.111111,1.833333,16.5,0.0,0.0,1.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14993,44994,11,11,11,0,0,11,11,0,0,0,0,11,0,0.0,0.0,0.0,0,0,2.000000,0.000000,0.0,0.0,0.0,2.000000,0.0,0.0
14994,44995,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
14995,44996,5,5,5,0,0,5,5,0,0,0,0,5,0,0.0,0.0,0.0,0,0,0.359223,0.000000,0.0,0.0,0.0,4.111111,0.0,0.0
14996,44997,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.541667,0.000000,0.0,0.0,0.0,6.500000,0.0,0.0


In [96]:
################################################# 케이스 선택 끝. 최종적으로 이후 사용할 변수는    ==> tmp3 <==    변수임. #################################################

## 2-4) err_code 변수에 대한 가공

In [97]:
# STEP 4. err_code 변수에 대한 전처리

################################################# 아래의 둘 중 하나의 케이스를 사용하여 작업하자. (위의 STEP1,2,3와 이어서 사용 할것.) #################################################

In [98]:
###############################################################################################################################################################
####################################################### case2. 전체 모델에 대해 errcode를 계산하는 경우  ######################################################
###############################################################################################################################################################

In [99]:
top50=list(train_err['errcode'].value_counts().head(50).index) # 주의!) 중요한점은, test전처리라 할 지라도, train 상위 50개를 기준으로 따라가야한다. 

# 그리고, errocde를 변환하는 함수를 만들고,
def cvt_errcode(x):
  if x in top50:
    return x
  else:
    return 'etc'

test_err['code_cvt']=test_err['errcode'].apply(cvt_errcode) # 적용.

In [100]:
dumi=pd.get_dummies(test_err[['user_id','model_nm','code_cvt']]) # model_nm과 errtype에 대해 더미변수 생성 (user_id 사용이뉴는 dumi변수에 그대로 가져가기 위함.)
tmp4=dumi.groupby(['user_id']).sum().astype(int).reset_index()

def convert_1(x): 
  # 숫자가 있으면 1, 없으면 0으로 바꾸는 함수.
  if x>0:
    return int(1)
  else:
    return int(0)

# model 변수에 대해 약 1600만개가 14999개로 압축되어 count되었기 때문에, 이를 1 또는 0으로 변환.
# tmp1에 아래의 데이터프레임을 저장함.
for i in range(9):
  tmp4['model_nm_model_'+str(i)]=tmp4['model_nm_model_'+str(i)].apply(convert_1)

del test_err['code_cvt']

display(tmp4)

,user_id,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,code_cvt_0,code_cvt_1,code_cvt_100,code_cvt_104,code_cvt_105,code_cvt_13,code_cvt_14,code_cvt_2,code_cvt_21,code_cvt_3,code_cvt_4,code_cvt_5,code_cvt_6,code_cvt_77,code_cvt_78,code_cvt_79,code_cvt_8.0,code_cvt_80,code_cvt_81,code_cvt_82,code_cvt_83,code_cvt_84,code_cvt_85,code_cvt_86,code_cvt_87,code_cvt_88,code_cvt_89,code_cvt_90,code_cvt_91,code_cvt_92,code_cvt_93,code_cvt_94,code_cvt_95,code_cvt_99,code_cvt_B-51049,code_cvt_B-A8002,code_cvt_H-51042,code_cvt_J-30021,code_cvt_NFANDROID2,code_cvt_Q-64002,code_cvt_S-61001,code_cvt_S-65002,code_cvt_UNKNOWN,code_cvt_V-21008,code_cvt_active,code_cvt_connection fail to establish,code_cvt_connection timeout,code_cvt_connectionterminated by local host,code_cvt_etc,code_cvt_standby,code_cvt_terminate by peer user
0,30000,0,1,1,0,0,0,0,0,0,90,1251,0,0,0,0,14,15,0,5,0,0,0,0,0,1,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,53,0,0,0,0,9,0,1,0,141,55,1107,0,0,4,0
1,30001,0,0,0,1,0,0,0,0,0,98,180,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0
2,30002,1,0,0,0,0,0,0,0,0,186,614,0,0,0,0,4,13,0,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,77,0,0,1,0,0,1,0,1,7,0,26,0,0,6,1
3,30003,1,0,0,0,0,0,0,0,0,91,147,0,0,0,0,11,1,0,1,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,106,0,0,8,0,0,1,0,0,1,0,1,0,0,0,0
4,30004,0,1,1,0,0,0,0,0,0,193,395,0,0,0,1,25,4,0,5,0,0,0,0,0,74,1,74,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,90,0,6,0,0,0,5,0,0,0,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14993,44994,0,1,0,0,0,0,0,0,0,139,624,0,0,0,5,26,14,0,2,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,87,0,12,0,0,0,0,7,197,0,0,0,0
14994,44995,1,0,0,0,0,0,0,0,0,105,370,0,0,0,2,6,13,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,0,0,0,0,0,0,0,0,0,0,0,0
14995,44996,1,0,0,0,0,0,0,0,0,251,698,0,0,0,3,4,529,0,573,6,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,104,4,44,0,1,11,2
14996,44997,1,0,0,0,0,0,0,0,0,61,11373,0,0,0,1,4,9,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,49,0,0,1,25,0,1655,1010,10449,1,6,20,1


In [101]:
# 43262번 처리 잊지말자.

id_43262 = pd.DataFrame(np.array([['43262']+[0]*60],dtype=int),columns=tmp4.columns) 
tmp4=tmp4.append(id_43262,ignore_index=True)
tmp4=tmp4.sort_values('user_id')
display(tmp4)

,user_id,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,code_cvt_0,code_cvt_1,code_cvt_100,code_cvt_104,code_cvt_105,code_cvt_13,code_cvt_14,code_cvt_2,code_cvt_21,code_cvt_3,code_cvt_4,code_cvt_5,code_cvt_6,code_cvt_77,code_cvt_78,code_cvt_79,code_cvt_8.0,code_cvt_80,code_cvt_81,code_cvt_82,code_cvt_83,code_cvt_84,code_cvt_85,code_cvt_86,code_cvt_87,code_cvt_88,code_cvt_89,code_cvt_90,code_cvt_91,code_cvt_92,code_cvt_93,code_cvt_94,code_cvt_95,code_cvt_99,code_cvt_B-51049,code_cvt_B-A8002,code_cvt_H-51042,code_cvt_J-30021,code_cvt_NFANDROID2,code_cvt_Q-64002,code_cvt_S-61001,code_cvt_S-65002,code_cvt_UNKNOWN,code_cvt_V-21008,code_cvt_active,code_cvt_connection fail to establish,code_cvt_connection timeout,code_cvt_connectionterminated by local host,code_cvt_etc,code_cvt_standby,code_cvt_terminate by peer user
0,30000,0,1,1,0,0,0,0,0,0,90,1251,0,0,0,0,14,15,0,5,0,0,0,0,0,1,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,53,0,0,0,0,9,0,1,0,141,55,1107,0,0,4,0
1,30001,0,0,0,1,0,0,0,0,0,98,180,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0
2,30002,1,0,0,0,0,0,0,0,0,186,614,0,0,0,0,4,13,0,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,77,0,0,1,0,0,1,0,1,7,0,26,0,0,6,1
3,30003,1,0,0,0,0,0,0,0,0,91,147,0,0,0,0,11,1,0,1,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,106,0,0,8,0,0,1,0,0,1,0,1,0,0,0,0
4,30004,0,1,1,0,0,0,0,0,0,193,395,0,0,0,1,25,4,0,5,0,0,0,0,0,74,1,74,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,90,0,6,0,0,0,5,0,0,0,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14993,44994,0,1,0,0,0,0,0,0,0,139,624,0,0,0,5,26,14,0,2,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,87,0,12,0,0,0,0,7,197,0,0,0,0
14994,44995,1,0,0,0,0,0,0,0,0,105,370,0,0,0,2,6,13,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,0,0,0,0,0,0,0,0,0,0,0,0
14995,44996,1,0,0,0,0,0,0,0,0,251,698,0,0,0,3,4,529,0,573,6,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,104,4,44,0,1,11,2
14996,44997,1,0,0,0,0,0,0,0,0,61,11373,0,0,0,1,4,9,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,49,0,0,1,25,0,1655,1010,10449,1,6,20,1


In [102]:
################################################# 케이스 선택 끝. 최종적으로 이후 사용할 변수는    ==> tmp4 <==    변수로 공통적임. #################################################

## 2-5) X_test dataset 생성

In [103]:
# Step5. 위의 4개 데이터(tmp1 ~ tmp4)를 종합하여 1개의 x_test 데이터를 만든다.

In [104]:
# 아래는 Case2에대한 merge이다. Case1에대한 merge는 해결을 못함... 서로의 데이터셋이 충돌하기때문..

In [105]:
display(tmp1)

,user_id,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
0,30000,0,1,1,0,0,0,0,0,0,0,0,0,0,62,1,1,0,0,0,16,16,0,3,67,37,9,3,0,1,0,983,1308,5,2,32,0,0,0,83,2,16,0,1,3,3,0,2,92,0,2
1,30001,0,0,0,1,0,0,0,0,0,0,0,3,98,3,4,4,0,0,0,12,16,4,6,53,55,0,0,0,0,0,0,0,0,0,24,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,30002,1,0,0,0,0,0,0,0,0,0,0,0,0,79,2,1,0,0,0,16,16,0,1,173,133,0,0,0,0,0,60,40,2,0,40,0,0,0,230,0,16,0,0,1,1,0,14,113,1,2
3,30003,1,0,0,0,0,0,0,0,0,0,0,0,0,107,2,2,0,0,0,4,5,1,7,8,12,0,1,0,1,0,3,2,0,0,5,2,2,0,134,0,4,16,3,1,1,0,0,40,8,0
4,30004,0,1,1,0,0,0,0,0,0,0,0,0,2,6,1,1,0,0,0,10,12,1,18,80,67,6,0,0,0,0,10,8,0,0,14,1,1,0,332,152,11,6,1,1,1,1,0,48,90,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14993,44994,0,1,0,0,0,0,0,0,0,0,0,0,2,12,5,5,0,0,0,16,19,3,9,123,93,12,0,0,0,0,197,204,0,0,31,0,0,0,179,2,16,0,0,0,0,0,0,100,87,0
14994,44995,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,14,15,1,4,149,98,0,0,0,0,0,0,0,0,0,1,0,0,0,183,0,14,1,0,1,1,0,0,12,17,0
14995,44996,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,12,13,1,3,129,90,2,0,0,0,0,138,165,0,0,48,0,0,0,163,0,12,1118,0,1,1,1,0,329,2,3
14996,44997,1,0,0,0,0,0,0,0,0,0,0,0,1,4,2,2,0,0,0,15,16,0,1,73,44,0,1,0,1,0,11127,13161,0,3,44,0,0,0,78,0,14,0,0,1,1,3,0,30,49,0


In [106]:
display(tmp2)

,user_id,cnt,valid_time
0,30000,808,23
1,30001,22,23
2,30002,67,4
3,30003,75,10
4,30004,85,11
...,...,...,...
14993,44994,90,15
14994,44995,35,24
14995,44996,430,15
14996,44997,2606,4


In [107]:
display(tmp3)

,user_id,quality_0_minus,quality_1_minus,quality_2_minus,quality_3_minus,quality_4_minus,quality_5_minus,quality_6_minus,quality_7_minus,quality_8_minus,quality_9_minus,quality_10_minus,quality_11_minus,quality_12_minus,quality_0_rest,quality_1_rest,quality_2_rest,quality_3_rest,quality_4_rest,quality_5_rest,quality_6_rest,quality_7_rest,quality_8_rest,quality_9_rest,quality_10_rest,quality_11_rest,quality_12_rest
0,30000,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.166667,0.416667,5.0,0.0,0.0,2.000000,0.0,0.0
1,30001,9,9,9,0,0,9,9,0,0,0,0,9,0,0.0,0.0,0.0,0,0,94.925926,0.000000,0.0,0.0,0.0,854.333333,0.0,0.0
2,30002,15,15,15,0,0,15,15,0,0,0,0,15,0,0.0,0.0,0.0,0,0,1.266667,0.000000,0.0,0.0,0.0,13.300000,0.0,0.0
3,30003,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.200000,3.666667,44.0,0.0,0.0,2.400000,0.0,0.0
4,30004,6,6,6,0,0,6,6,0,0,0,0,6,0,0.0,0.0,0.0,0,0,0.111111,1.833333,16.5,0.0,0.0,1.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14993,44994,11,11,11,0,0,11,11,0,0,0,0,11,0,0.0,0.0,0.0,0,0,2.000000,0.000000,0.0,0.0,0.0,2.000000,0.0,0.0
14994,44995,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
14995,44996,5,5,5,0,0,5,5,0,0,0,0,5,0,0.0,0.0,0.0,0,0,0.359223,0.000000,0.0,0.0,0.0,4.111111,0.0,0.0
14996,44997,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.541667,0.000000,0.0,0.0,0.0,6.500000,0.0,0.0


In [108]:
display(tmp4)

,user_id,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,code_cvt_0,code_cvt_1,code_cvt_100,code_cvt_104,code_cvt_105,code_cvt_13,code_cvt_14,code_cvt_2,code_cvt_21,code_cvt_3,code_cvt_4,code_cvt_5,code_cvt_6,code_cvt_77,code_cvt_78,code_cvt_79,code_cvt_8.0,code_cvt_80,code_cvt_81,code_cvt_82,code_cvt_83,code_cvt_84,code_cvt_85,code_cvt_86,code_cvt_87,code_cvt_88,code_cvt_89,code_cvt_90,code_cvt_91,code_cvt_92,code_cvt_93,code_cvt_94,code_cvt_95,code_cvt_99,code_cvt_B-51049,code_cvt_B-A8002,code_cvt_H-51042,code_cvt_J-30021,code_cvt_NFANDROID2,code_cvt_Q-64002,code_cvt_S-61001,code_cvt_S-65002,code_cvt_UNKNOWN,code_cvt_V-21008,code_cvt_active,code_cvt_connection fail to establish,code_cvt_connection timeout,code_cvt_connectionterminated by local host,code_cvt_etc,code_cvt_standby,code_cvt_terminate by peer user
0,30000,0,1,1,0,0,0,0,0,0,90,1251,0,0,0,0,14,15,0,5,0,0,0,0,0,1,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,53,0,0,0,0,9,0,1,0,141,55,1107,0,0,4,0
1,30001,0,0,0,1,0,0,0,0,0,98,180,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0
2,30002,1,0,0,0,0,0,0,0,0,186,614,0,0,0,0,4,13,0,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,77,0,0,1,0,0,1,0,1,7,0,26,0,0,6,1
3,30003,1,0,0,0,0,0,0,0,0,91,147,0,0,0,0,11,1,0,1,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,106,0,0,8,0,0,1,0,0,1,0,1,0,0,0,0
4,30004,0,1,1,0,0,0,0,0,0,193,395,0,0,0,1,25,4,0,5,0,0,0,0,0,74,1,74,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,90,0,6,0,0,0,5,0,0,0,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14993,44994,0,1,0,0,0,0,0,0,0,139,624,0,0,0,5,26,14,0,2,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,87,0,12,0,0,0,0,7,197,0,0,0,0
14994,44995,1,0,0,0,0,0,0,0,0,105,370,0,0,0,2,6,13,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,0,0,0,0,0,0,0,0,0,0,0,0
14995,44996,1,0,0,0,0,0,0,0,0,251,698,0,0,0,3,4,529,0,573,6,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,104,4,44,0,1,11,2
14996,44997,1,0,0,0,0,0,0,0,0,61,11373,0,0,0,1,4,9,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,49,0,0,1,25,0,1655,1010,10449,1,6,20,1


In [109]:
X = pd.merge(tmp1,tmp2,on=['user_id'],how='left')
X = pd.merge(X,tmp3,on=['user_id'],how='left')
X = pd.merge(X,tmp4,on=['user_id','model_nm_model_0','model_nm_model_1','model_nm_model_2','model_nm_model_3','model_nm_model_4','model_nm_model_5','model_nm_model_6','model_nm_model_7','model_nm_model_8'],how='left')

In [110]:
display(X)

,user_id,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,...,code_cvt_5,code_cvt_6,code_cvt_77,code_cvt_78,code_cvt_79,code_cvt_8.0,code_cvt_80,code_cvt_81,code_cvt_82,code_cvt_83,code_cvt_84,code_cvt_85,code_cvt_86,code_cvt_87,code_cvt_88,code_cvt_89,code_cvt_90,code_cvt_91,code_cvt_92,code_cvt_93,code_cvt_94,code_cvt_95,code_cvt_99,code_cvt_B-51049,code_cvt_B-A8002,code_cvt_H-51042,code_cvt_J-30021,code_cvt_NFANDROID2,code_cvt_Q-64002,code_cvt_S-61001,code_cvt_S-65002,code_cvt_UNKNOWN,code_cvt_V-21008,code_cvt_active,code_cvt_connection fail to establish,code_cvt_connection timeout,code_cvt_connectionterminated by local host,code_cvt_etc,code_cvt_standby,code_cvt_terminate by peer user
0,30000,0,1,1,0,0,0,0,0,0,0,0,0,0,62,1,1,0,0,0,16,16,0,3,67,37,9,3,0,1,0,983,1308,5,2,32,0,0,0,83,...,0,0,0,0,1,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,53,0,0,0,0,9,0,1,0,141,55,1107,0,0,4,0
1,30001,0,0,0,1,0,0,0,0,0,0,0,3,98,3,4,4,0,0,0,12,16,4,6,53,55,0,0,0,0,0,0,0,0,0,24,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0
2,30002,1,0,0,0,0,0,0,0,0,0,0,0,0,79,2,1,0,0,0,16,16,0,1,173,133,0,0,0,0,0,60,40,2,0,40,0,0,0,230,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,77,0,0,1,0,0,1,0,1,7,0,26,0,0,6,1
3,30003,1,0,0,0,0,0,0,0,0,0,0,0,0,107,2,2,0,0,0,4,5,1,7,8,12,0,1,0,1,0,3,2,0,0,5,2,2,0,134,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,106,0,0,8,0,0,1,0,0,1,0,1,0,0,0,0
4,30004,0,1,1,0,0,0,0,0,0,0,0,0,2,6,1,1,0,0,0,10,12,1,18,80,67,6,0,0,0,0,10,8,0,0,14,1,1,0,332,...,0,0,0,0,74,1,74,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,90,0,6,0,0,0,5,0,0,0,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,44994,0,1,0,0,0,0,0,0,0,0,0,0,2,12,5,5,0,0,0,16,19,3,9,123,93,12,0,0,0,0,197,204,0,0,31,0,0,0,179,...,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,87,0,12,0,0,0,0,7,197,0,0,0,0
14995,44995,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,14,15,1,4,149,98,0,0,0,0,0,0,0,0,0,1,0,0,0,183,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,0,0,0,0,0,0,0,0,0,0,0,0
14996,44996,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,12,13,1,3,129,90,2,0,0,0,0,138,165,0,0,48,0,0,0,163,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,104,4,44,0,1,11,2
14997,44997,1,0,0,0,0,0,0,0,0,0,0,0,1,4,2,2,0,0,0,15,16,0,1,73,44,0,1,0,1,0,11127,13161,0,3,44,0,0,0,78,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,49,0,0,1,25,0,1655,1010,10449,1,6,20,1


In [111]:
# X저장
del X['user_id']
np.savetxt(os.path.join(current_day,'x_test.csv'),X)
# pandas
X.to_csv(os.path.join(current_day,'x_test_df.csv'),index=False)